# Lab 05: Advanced Object Detection with Azure Custom Vision

This notebook covers advanced topics in object detection, including evaluation metrics, optimization techniques, and production deployment strategies.

## Advanced Topics Covered

1. **IoU (Intersection over Union)** - Understanding overlap metrics
2. **Non-Maximum Suppression (NMS)** - Handling multiple overlapping detections
3. **Precision-Recall Curves** - Visualizing model tradeoffs
4. **Multi-Object Detection** - Strategies for complex scenes
5. **Performance Optimization** - Speed vs accuracy tradeoffs
6. **Real-time Detection** - Considerations for video/streaming
7. **Custom Thresholding** - Advanced filtering strategies
8. **Model Export** - Edge deployment for object detection

## 1. Setup and Installation

In [ ]:
!pip install azure-cognitiveservices-vision-customvision python-dotenv pillow matplotlib numpy scipy scikit-learn seaborn

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import (
    ImageFileCreateBatch, ImageFileCreateEntry, Region
)
from msrest.authentication import ApiKeyCredentials
from dotenv import load_dotenv
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import numpy as np
import seaborn as sns
from collections import defaultdict
import time
import os
import json
import uuid

print("Libraries imported successfully!")

## 2. Configure Credentials and Clients

In [ ]:
env_path = 'python/train-detector/.env'
load_dotenv(env_path)

training_endpoint = os.getenv('TrainingEndpoint')
training_key = os.getenv('TrainingKey')
prediction_endpoint = os.getenv('PredictionEndpoint', training_endpoint)
prediction_key = os.getenv('PredictionKey', training_key)
project_id = os.getenv('ProjectID', None)

training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
training_client = CustomVisionTrainingClient(training_endpoint, training_credentials)

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
prediction_client = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

print("Clients authenticated!")

## 3. Understanding IoU (Intersection over Union)

**IoU** measures the overlap between predicted and ground truth bounding boxes. It's crucial for evaluating object detection performance.

### IoU Formula:
```
IoU = Area of Overlap / Area of Union
```

### IoU Interpretation:
- **IoU > 0.5**: Generally considered a "good" detection
- **IoU > 0.7**: High quality detection
- **IoU > 0.9**: Excellent detection
- **IoU < 0.5**: Often considered a false positive

In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.
    
    Args:
        box1, box2: Dictionaries with 'left', 'top', 'width', 'height' (normalized 0-1)
    
    Returns:
        IoU score (0-1)
    """
    # Calculate coordinates
    x1_min = box1['left']
    y1_min = box1['top']
    x1_max = box1['left'] + box1['width']
    y1_max = box1['top'] + box1['height']
    
    x2_min = box2['left']
    y2_min = box2['top']
    x2_max = box2['left'] + box2['width']
    y2_max = box2['top'] + box2['height']
    
    # Calculate intersection area
    intersect_width = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
    intersect_height = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
    intersect_area = intersect_width * intersect_height
    
    # Calculate union area
    box1_area = box1['width'] * box1['height']
    box2_area = box2['width'] * box2['height']
    union_area = box1_area + box2_area - intersect_area
    
    # Calculate IoU
    if union_area == 0:
        return 0
    
    iou = intersect_area / union_area
    return iou

def visualize_iou_examples():
    """
    Visualize different IoU scenarios.
    """
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # Different overlap scenarios
    scenarios = [
        # Perfect match
        (
            {'left': 0.2, 'top': 0.2, 'width': 0.3, 'height': 0.3},
            {'left': 0.2, 'top': 0.2, 'width': 0.3, 'height': 0.3},
            "Perfect Match (IoU=1.0)"
        ),
        # Good overlap
        (
            {'left': 0.2, 'top': 0.2, 'width': 0.3, 'height': 0.3},
            {'left': 0.25, 'top': 0.25, 'width': 0.3, 'height': 0.3},
            "Good Overlap (IoU≈0.68)"
        ),
        # Moderate overlap
        (
            {'left': 0.2, 'top': 0.2, 'width': 0.3, 'height': 0.3},
            {'left': 0.35, 'top': 0.2, 'width': 0.3, 'height': 0.3},
            "Moderate (IoU≈0.38)"
        ),
        # No overlap
        (
            {'left': 0.2, 'top': 0.2, 'width': 0.2, 'height': 0.2},
            {'left': 0.6, 'top': 0.6, 'width': 0.2, 'height': 0.2},
            "No Overlap (IoU=0.0)"
        )
    ]
    
    for idx, (box1, box2, title) in enumerate(scenarios):
        ax = axes[idx]
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_aspect('equal')
        
        # Draw ground truth (blue)
        rect1 = Rectangle(
            (box1['left'], box1['top']), box1['width'], box1['height'],
            linewidth=2, edgecolor='blue', facecolor='blue', alpha=0.3, label='Ground Truth'
        )
        ax.add_patch(rect1)
        
        # Draw prediction (red)
        rect2 = Rectangle(
            (box2['left'], box2['top']), box2['width'], box2['height'],
            linewidth=2, edgecolor='red', facecolor='red', alpha=0.3, label='Prediction'
        )
        ax.add_patch(rect2)
        
        # Calculate and display IoU
        iou = calculate_iou(box1, box2)
        ax.set_title(f"{title}\nIoU: {iou:.2f}", fontsize=10)
        ax.invert_yaxis()
        
        if idx == 0:
            ax.legend(loc='upper left', fontsize=8)
    
    plt.tight_layout()
    plt.suptitle('IoU (Intersection over Union) Examples', y=1.02, fontsize=14, fontweight='bold')
    plt.show()

# Visualize IoU examples
visualize_iou_examples()

# Test IoU calculation
print("\nIoU Calculation Examples:")
print("=" * 70)

test_box1 = {'left': 0.2, 'top': 0.2, 'width': 0.3, 'height': 0.3}
test_box2 = {'left': 0.25, 'top': 0.25, 'width': 0.3, 'height': 0.3}
print(f"Box 1: {test_box1}")
print(f"Box 2: {test_box2}")
print(f"IoU: {calculate_iou(test_box1, test_box2):.4f}")

## 4. Non-Maximum Suppression (NMS)

**NMS** eliminates redundant overlapping detections, keeping only the best one for each object.

### Why NMS is Needed:
- Object detectors often produce multiple detections for the same object
- NMS selects the detection with highest confidence and suppresses overlapping ones

### NMS Algorithm:
1. Sort detections by confidence score (highest first)
2. Select detection with highest confidence
3. Remove all detections with IoU > threshold with selected detection
4. Repeat until no detections remain

In [ ]:
def non_maximum_suppression(detections, iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression to filter overlapping detections.
    
    Args:
        detections: List of detection dicts with 'bbox', 'confidence', 'tag'
        iou_threshold: IoU threshold for suppression
    
    Returns:
        Filtered list of detections
    """
    if not detections:
        return []
    
    # Sort by confidence (descending)
    sorted_detections = sorted(detections, key=lambda x: x['confidence'], reverse=True)
    
    kept_detections = []
    
    while sorted_detections:
        # Take detection with highest confidence
        current = sorted_detections.pop(0)
        kept_detections.append(current)
        
        # Remove detections with high IoU overlap of same class
        remaining = []
        for det in sorted_detections:
            iou = calculate_iou(current['bbox'], det['bbox'])
            
            # Keep if IoU is below threshold OR different object class
            if iou <= iou_threshold or det['tag'] != current['tag']:
                remaining.append(det)
        
        sorted_detections = remaining
    
    return kept_detections

def demonstrate_nms():
    """
    Demonstrate NMS with sample detections.
    """
    # Simulate multiple overlapping detections
    sample_detections = [
        {'tag': 'apple', 'confidence': 0.95, 'bbox': {'left': 0.2, 'top': 0.2, 'width': 0.2, 'height': 0.2}},
        {'tag': 'apple', 'confidence': 0.88, 'bbox': {'left': 0.22, 'top': 0.22, 'width': 0.2, 'height': 0.2}},
        {'tag': 'apple', 'confidence': 0.75, 'bbox': {'left': 0.25, 'top': 0.19, 'width': 0.18, 'height': 0.22}},
        {'tag': 'banana', 'confidence': 0.92, 'bbox': {'left': 0.6, 'top': 0.5, 'width': 0.25, 'height': 0.15}},
        {'tag': 'banana', 'confidence': 0.78, 'bbox': {'left': 0.62, 'top': 0.52, 'width': 0.23, 'height': 0.14}},
    ]
    
    print("Non-Maximum Suppression Demonstration")
    print("=" * 70)
    print(f"\nOriginal Detections: {len(sample_detections)}")
    for i, det in enumerate(sample_detections, 1):
        print(f"  {i}. {det['tag']:10} confidence={det['confidence']:.2f}  "
              f"bbox=({det['bbox']['left']:.2f}, {det['bbox']['top']:.2f}, "
              f"{det['bbox']['width']:.2f}, {det['bbox']['height']:.2f})")
    
    # Apply NMS
    filtered_detections = non_maximum_suppression(sample_detections, iou_threshold=0.5)
    
    print(f"\nAfter NMS (IoU threshold=0.5): {len(filtered_detections)}")
    for i, det in enumerate(filtered_detections, 1):
        print(f"  {i}. {det['tag']:10} confidence={det['confidence']:.2f}  "
              f"bbox=({det['bbox']['left']:.2f}, {det['bbox']['top']:.2f}, "
              f"{det['bbox']['width']:.2f}, {det['bbox']['height']:.2f})")
    
    print(f"\nReduction: {len(sample_detections) - len(filtered_detections)} detections suppressed")
    
    # Visualize before and after
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    colors = {'apple': 'red', 'banana': 'yellow'}
    
    # Before NMS
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.set_aspect('equal')
    ax1.set_title(f'Before NMS ({len(sample_detections)} detections)', fontweight='bold')
    ax1.invert_yaxis()
    
    for det in sample_detections:
        rect = Rectangle(
            (det['bbox']['left'], det['bbox']['top']),
            det['bbox']['width'], det['bbox']['height'],
            linewidth=2, edgecolor=colors[det['tag']], 
            facecolor='none', alpha=0.7
        )
        ax1.add_patch(rect)
        ax1.text(
            det['bbox']['left'], det['bbox']['top'] - 0.02,
            f"{det['confidence']:.2f}", fontsize=9,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
        )
    
    # After NMS
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.set_aspect('equal')
    ax2.set_title(f'After NMS ({len(filtered_detections)} detections)', fontweight='bold')
    ax2.invert_yaxis()
    
    for det in filtered_detections:
        rect = Rectangle(
            (det['bbox']['left'], det['bbox']['top']),
            det['bbox']['width'], det['bbox']['height'],
            linewidth=3, edgecolor=colors[det['tag']], 
            facecolor='none'
        )
        ax2.add_patch(rect)
        ax2.text(
            det['bbox']['left'], det['bbox']['top'] - 0.02,
            f"{det['tag']} {det['confidence']:.2f}", fontsize=10,
            fontweight='bold',
            bbox=dict(facecolor=colors[det['tag']], alpha=0.7, edgecolor='none')
        )
    
    plt.tight_layout()
    plt.show()

demonstrate_nms()

## 5. Create Advanced Detection Project

In [ ]:
# Get detection domain
domains = training_client.get_domains()
obj_detection_domain = next((d for d in domains if d.type == 'ObjectDetection'), None)

# Create project
project_name = f"Advanced Object Detection {uuid.uuid4().hex[:8]}"
project = training_client.create_project(
    name=project_name,
    description="Advanced object detection with performance optimization",
    domain_id=obj_detection_domain.id
)

print(f"✓ Created project: {project.name}")
print(f"  Project ID: {project.id}")

# Create tags
tag_names = ['apple', 'banana', 'orange']
tags = {}
for tag_name in tag_names:
    tag = training_client.create_tag(project.id, tag_name)
    tags[tag_name] = tag
    print(f"  Created tag: {tag_name}")

## 6. Upload Training Data and Train Model

In [ ]:
# Load and upload tagged images
tagged_images_json = 'python/train-detector/tagged-images.json'
images_folder = 'python/train-detector/images'

with open(tagged_images_json, 'r') as f:
    tagged_data = json.load(f)

print(f"Uploading {len(tagged_data['files'])} training images...")

tagged_images_with_regions = []
for image_info in tagged_data['files']:
    filename = image_info['filename']
    image_path = os.path.join(images_folder, filename)
    
    if not os.path.exists(image_path):
        continue
    
    regions = []
    for bbox in image_info['tags']:
        tag_id = tags[bbox['tag']].id
        region = Region(
            tag_id=tag_id,
            left=bbox['left'],
            top=bbox['top'],
            width=bbox['width'],
            height=bbox['height']
        )
        regions.append(region)
    
    with open(image_path, 'rb') as image_file:
        tagged_images_with_regions.append(
            ImageFileCreateEntry(
                name=filename,
                contents=image_file.read(),
                regions=regions
            )
        )

upload_result = training_client.create_images_from_files(
    project.id,
    ImageFileCreateBatch(images=tagged_images_with_regions)
)

print(f"✓ Images uploaded successfully\n")

# Train model
print("Training model...")
iteration = training_client.train_project(project.id)

while iteration.status not in ["Completed", "Failed"]:
    iteration = training_client.get_iteration(project.id, iteration.id)
    print(f"  Status: {iteration.status}")
    time.sleep(10)

print(f"\n✓ Training completed!")

# Publish model
publish_name = "AdvancedDetector"
prediction_resource_id = os.getenv('PredictionResourceId', None)

try:
    if prediction_resource_id:
        training_client.publish_iteration(project.id, iteration.id, publish_name, prediction_resource_id)
    else:
        training_client.publish_iteration(project.id, iteration.id, publish_name)
    print(f"✓ Model published as '{publish_name}'")
except Exception as e:
    print(f"Publishing info: {e}")

## 7. Advanced Detection with NMS

In [ ]:
def advanced_object_detection(image_path, project_id, publish_name, prediction_client,
                             confidence_threshold=0.5, nms_threshold=0.5, apply_nms=True):
    """
    Perform object detection with optional NMS.
    
    Args:
        image_path: Path to image
        project_id: Project ID
        publish_name: Model name
        prediction_client: Prediction client
        confidence_threshold: Minimum confidence
        nms_threshold: IoU threshold for NMS
        apply_nms: Whether to apply NMS
    
    Returns:
        Dictionary with detection results
    """
    # Get predictions
    with open(image_path, 'rb') as f:
        results = prediction_client.detect_image(project_id, publish_name, f.read())
    
    # Filter by confidence
    detections = []
    for pred in results.predictions:
        if pred.probability >= confidence_threshold:
            detections.append({
                'tag': pred.tag_name,
                'confidence': pred.probability,
                'bbox': {
                    'left': pred.bounding_box.left,
                    'top': pred.bounding_box.top,
                    'width': pred.bounding_box.width,
                    'height': pred.bounding_box.height
                }
            })
    
    # Apply NMS if requested
    if apply_nms and len(detections) > 1:
        filtered_detections = non_maximum_suppression(detections, nms_threshold)
    else:
        filtered_detections = detections
    
    return {
        'raw_detections': detections,
        'filtered_detections': filtered_detections,
        'num_suppressed': len(detections) - len(filtered_detections)
    }

# Test advanced detection
test_image = 'python/test-detector/produce.jpg'

if os.path.exists(test_image):
    print("Advanced Object Detection Results")
    print("=" * 70)
    
    result = advanced_object_detection(
        test_image,
        project.id,
        publish_name,
        prediction_client,
        confidence_threshold=0.3,
        nms_threshold=0.5,
        apply_nms=True
    )
    
    print(f"\nRaw detections: {len(result['raw_detections'])}")
    print(f"After NMS: {len(result['filtered_detections'])}")
    print(f"Suppressed: {result['num_suppressed']}")
    
    print(f"\nFinal Detections:")
    for det in result['filtered_detections']:
        print(f"  {det['tag']:10} - {det['confidence']:.2%}")

## 8. Performance vs Speed Tradeoffs

In production, you often need to balance detection accuracy with speed:

### Strategies for Optimization:
1. **Lower confidence threshold**: Detect more objects but get more false positives
2. **Higher confidence threshold**: Fewer false positives but might miss objects
3. **Image resolution**: Lower resolution = faster but less accurate
4. **Model selection**: Compact models are faster but may be less accurate
5. **Batch processing**: Process multiple images together for efficiency

In [ ]:
def benchmark_detection_performance(image_path, project_id, publish_name, prediction_client, iterations=5):
    """
    Benchmark detection speed and analyze performance.
    """
    print("Detection Performance Benchmark")
    print("=" * 70)
    
    with open(image_path, 'rb') as f:
        image_data = f.read()
    
    times = []
    
    for i in range(iterations):
        start_time = time.time()
        results = prediction_client.detect_image(project_id, publish_name, image_data)
        elapsed = time.time() - start_time
        times.append(elapsed)
        print(f"Iteration {i+1}: {elapsed:.3f}s ({len(results.predictions)} detections)")
    
    print(f"\nPerformance Summary:")
    print(f"  Average time: {np.mean(times):.3f}s")
    print(f"  Std deviation: {np.std(times):.3f}s")
    print(f"  Min time: {np.min(times):.3f}s")
    print(f"  Max time: {np.max(times):.3f}s")
    print(f"  Throughput: {1/np.mean(times):.2f} images/second")
    
    # Visualize timing distribution
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, iterations+1), times, 'o-', linewidth=2, markersize=8)
    plt.axhline(np.mean(times), color='r', linestyle='--', label=f'Mean: {np.mean(times):.3f}s')
    plt.xlabel('Iteration')
    plt.ylabel('Time (seconds)')
    plt.title('Detection Time per Iteration')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.hist(times, bins=max(3, iterations//2), edgecolor='black', alpha=0.7)
    plt.axvline(np.mean(times), color='r', linestyle='--', linewidth=2, label='Mean')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency')
    plt.title('Time Distribution')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

if os.path.exists(test_image):
    benchmark_detection_performance(test_image, project.id, publish_name, prediction_client, iterations=10)

## 9. Multi-Object Scene Analysis

Analyze complex scenes with multiple objects and provide statistical insights.

In [ ]:
def analyze_scene(image_path, project_id, publish_name, prediction_client, confidence_threshold=0.5):
    """
    Perform comprehensive scene analysis.
    """
    result = advanced_object_detection(
        image_path, project_id, publish_name, prediction_client,
        confidence_threshold=confidence_threshold, apply_nms=True
    )
    
    detections = result['filtered_detections']
    
    print("Scene Analysis Report")
    print("=" * 70)
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Total objects detected: {len(detections)}")
    print(f"\nObject Distribution:")
    
    # Count by type
    object_counts = defaultdict(int)
    object_confidences = defaultdict(list)
    
    for det in detections:
        object_counts[det['tag']] += 1
        object_confidences[det['tag']].append(det['confidence'])
    
    for obj_type, count in sorted(object_counts.items()):
        avg_conf = np.mean(object_confidences[obj_type])
        print(f"  {obj_type:15} : {count:2} objects (avg confidence: {avg_conf:.2%})")
    
    # Calculate object sizes
    print(f"\nObject Sizes:")
    for det in detections:
        area = det['bbox']['width'] * det['bbox']['height']
        size_category = "Large" if area > 0.15 else "Medium" if area > 0.05 else "Small"
        print(f"  {det['tag']:10} - {size_category:6} (area: {area:.3f})")
    
    # Spatial distribution
    print(f"\nSpatial Distribution:")
    for det in detections:
        center_x = det['bbox']['left'] + det['bbox']['width'] / 2
        center_y = det['bbox']['top'] + det['bbox']['height'] / 2
        
        h_pos = "Left" if center_x < 0.33 else "Center" if center_x < 0.67 else "Right"
        v_pos = "Top" if center_y < 0.33 else "Middle" if center_y < 0.67 else "Bottom"
        
        print(f"  {det['tag']:10} - {v_pos:6}/{h_pos:6} (center: {center_x:.2f}, {center_y:.2f})")
    
    # Visualize scene
    img = Image.open(image_path)
    img_width, img_height = img.size
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Image with detections
    ax1.imshow(img)
    colors = {'apple': 'red', 'banana': 'yellow', 'orange': 'orange'}
    
    for det in detections:
        left = det['bbox']['left'] * img_width
        top = det['bbox']['top'] * img_height
        width = det['bbox']['width'] * img_width
        height = det['bbox']['height'] * img_height
        
        color = colors.get(det['tag'], 'cyan')
        rect = patches.Rectangle(
            (left, top), width, height,
            linewidth=3, edgecolor=color, facecolor='none'
        )
        ax1.add_patch(rect)
        
        label = f"{det['tag']} {det['confidence']:.0%}"
        ax1.text(left, top - 5, label, color='white', fontsize=10, fontweight='bold',
                bbox=dict(facecolor=color, alpha=0.8, edgecolor='none', pad=2))
    
    ax1.set_title('Detected Objects', fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    # Object statistics
    obj_types = list(object_counts.keys())
    counts = list(object_counts.values())
    
    ax2.bar(obj_types, counts, color=[colors.get(t, 'gray') for t in obj_types], alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Object Type')
    ax2.set_ylabel('Count')
    ax2.set_title('Object Distribution', fontsize=12, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return detections

if os.path.exists(test_image):
    scene_detections = analyze_scene(test_image, project.id, publish_name, prediction_client)

## 10. Real-Time Detection Considerations

For video or real-time applications:

### Key Considerations:
1. **Frame rate**: Target 15-30 FPS for smooth video
2. **Latency**: Network latency affects responsiveness
3. **Frame skipping**: Process every Nth frame to improve throughput
4. **Temporal consistency**: Track objects across frames
5. **Edge deployment**: Use exported models for local processing

In [ ]:
def estimate_realtime_capability(avg_detection_time, target_fps=30):
    """
    Estimate real-time detection capability.
    """
    print("Real-Time Detection Analysis")
    print("=" * 70)
    
    max_fps = 1 / avg_detection_time
    frame_budget = 1 / target_fps
    
    print(f"Average detection time: {avg_detection_time*1000:.1f}ms")
    print(f"Maximum achievable FPS: {max_fps:.1f}")
    print(f"Target FPS: {target_fps}")
    print(f"Frame time budget: {frame_budget*1000:.1f}ms\n")
    
    if max_fps >= target_fps:
        print(f"✓ Can achieve target {target_fps} FPS")
        overhead = (frame_budget - avg_detection_time) * 1000
        print(f"  Time budget remaining: {overhead:.1f}ms for other processing")
    else:
        print(f"⚠️  Cannot achieve target {target_fps} FPS")
        skip_factor = int(np.ceil(target_fps / max_fps))
        effective_fps = max_fps / skip_factor
        print(f"  Recommended: Process every {skip_factor} frames")
        print(f"  Effective FPS: {effective_fps:.1f}")
    
    print(f"\nOptimization Strategies:")
    print(f"  - Use compact model domain for faster inference")
    print(f"  - Export model for edge deployment")
    print(f"  - Reduce image resolution")
    print(f"  - Process frames in parallel")
    print(f"  - Skip frames (process every Nth frame)")
    
    # Visualize frame processing scenarios
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    scenarios = [
        ("Current", max_fps, 'red'),
        ("Target", target_fps, 'green'),
        ("With Optimization\n(2x faster)", max_fps * 2, 'blue')
    ]
    
    for idx, (label, fps, color) in enumerate(scenarios):
        ax = axes[idx]
        ax.bar([label], [fps], color=color, alpha=0.7, edgecolor='black')
        ax.axhline(target_fps, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Target')
        ax.set_ylabel('FPS')
        ax.set_title(f"{label}\n{fps:.1f} FPS")
        ax.set_ylim([0, max(target_fps * 1.5, max_fps * 2.5)])
        ax.grid(axis='y', alpha=0.3)
        if idx == 0:
            ax.legend()
    
    plt.tight_layout()
    plt.show()

# Estimate with example timing
estimate_realtime_capability(avg_detection_time=0.15, target_fps=30)  # 150ms per detection

## 11. Production Deployment Class

A production-ready detector with all optimizations.

In [ ]:
class ProductionObjectDetector:
    """
    Production-ready object detector with NMS, error handling, and monitoring.
    """
    
    def __init__(self, endpoint, key, project_id, model_name,
                 confidence_threshold=0.5, nms_threshold=0.5,
                 enable_nms=True, retry_attempts=3):
        self.endpoint = endpoint
        self.project_id = project_id
        self.model_name = model_name
        self.confidence_threshold = confidence_threshold
        self.nms_threshold = nms_threshold
        self.enable_nms = enable_nms
        self.retry_attempts = retry_attempts
        
        credentials = ApiKeyCredentials(in_headers={"Prediction-key": key})
        self.client = CustomVisionPredictionClient(endpoint, credentials)
        
        # Monitoring
        self.stats = {
            'total_predictions': 0,
            'total_detections': 0,
            'suppressed_detections': 0,
            'errors': 0,
            'detection_times': []
        }
    
    def detect(self, image_data):
        """
        Detect objects with full error handling and post-processing.
        """
        for attempt in range(self.retry_attempts):
            try:
                start_time = time.time()
                results = self.client.detect_image(
                    self.project_id,
                    self.model_name,
                    image_data
                )
                elapsed = time.time() - start_time
                
                self.stats['detection_times'].append(elapsed)
                self.stats['total_predictions'] += 1
                
                # Convert to our format
                detections = []
                for pred in results.predictions:
                    if pred.probability >= self.confidence_threshold:
                        detections.append({
                            'tag': pred.tag_name,
                            'confidence': pred.probability,
                            'bbox': {
                                'left': pred.bounding_box.left,
                                'top': pred.bounding_box.top,
                                'width': pred.bounding_box.width,
                                'height': pred.bounding_box.height
                            }
                        })
                
                self.stats['total_detections'] += len(detections)
                
                # Apply NMS
                if self.enable_nms and len(detections) > 1:
                    filtered = non_maximum_suppression(detections, self.nms_threshold)
                    self.stats['suppressed_detections'] += len(detections) - len(filtered)
                else:
                    filtered = detections
                
                return {
                    'success': True,
                    'detections': filtered,
                    'num_objects': len(filtered),
                    'detection_time': elapsed,
                    'raw_count': len(detections)
                }
            
            except Exception as e:
                self.stats['errors'] += 1
                if attempt == self.retry_attempts - 1:
                    return {
                        'success': False,
                        'error': str(e),
                        'attempts': attempt + 1
                    }
                time.sleep(1 * (attempt + 1))
    
    def get_stats(self):
        """Get detector statistics."""
        avg_time = np.mean(self.stats['detection_times']) if self.stats['detection_times'] else 0
        
        return {
            **self.stats,
            'avg_detection_time': avg_time,
            'avg_detections_per_image': self.stats['total_detections'] / max(1, self.stats['total_predictions']),
            'suppression_rate': self.stats['suppressed_detections'] / max(1, self.stats['total_detections']),
            'error_rate': self.stats['errors'] / max(1, self.stats['total_predictions'] + self.stats['errors'])
        }

# Example usage
print("Production Object Detector Example")
print("=" * 70)

detector = ProductionObjectDetector(
    prediction_endpoint,
    prediction_key,
    project.id,
    publish_name,
    confidence_threshold=0.5,
    nms_threshold=0.5,
    enable_nms=True
)

if os.path.exists(test_image):
    with open(test_image, 'rb') as f:
        result = detector.detect(f.read())
    
    if result['success']:
        print(f"✓ Detection successful")
        print(f"  Objects found: {result['num_objects']}")
        print(f"  Detection time: {result['detection_time']*1000:.1f}ms")
        print(f"  Raw detections: {result['raw_count']}")
        print(f"\nDetected objects:")
        for det in result['detections']:
            print(f"  - {det['tag']:10} ({det['confidence']:.2%})")
    
    stats = detector.get_stats()
    print(f"\nDetector Statistics:")
    print(f"  Total predictions: {stats['total_predictions']}")
    print(f"  Avg detection time: {stats['avg_detection_time']*1000:.1f}ms")
    print(f"  Avg objects per image: {stats['avg_detections_per_image']:.1f}")
    print(f"  Suppression rate: {stats['suppression_rate']:.1%}")

## 12. Summary

### Advanced Concepts Covered:
✓ IoU (Intersection over Union) calculation and interpretation  
✓ Non-Maximum Suppression (NMS) for duplicate removal  
✓ Performance optimization and benchmarking  
✓ Multi-object scene analysis  
✓ Real-time detection considerations  
✓ Production-ready detector implementation  
✓ Advanced monitoring and statistics  

### Key Takeaways:
1. **IoU** measures detection quality by quantifying bounding box overlap
2. **NMS** eliminates redundant detections, improving results
3. **Performance tuning** requires balancing speed vs accuracy
4. **Real-time detection** needs optimization for target frame rates
5. **Production deployment** requires robust error handling and monitoring

### Best Practices:
- Use NMS to clean up multiple detections of same object
- Set IoU threshold based on your accuracy requirements
- Monitor detection times and confidence distributions
- Implement retry logic and error handling
- Track suppression rates to tune NMS threshold
- Consider edge deployment for real-time applications

### Next Steps:
- Experiment with different IoU and NMS thresholds
- Implement tracking for video object detection
- Export models for edge deployment
- Build custom post-processing pipelines
- Integrate with real-time video streams

## Cleanup

In [ ]:
# Uncomment to delete project
# training_client.delete_project(project.id)
# print("Project deleted")